In [1]:
# State

In [1]:
import numpy as np

import quara.objects.qoperation_typical as qt
from quara.objects.composite_system_typical import generate_composite_system

from quara.loss_function.weighted_probability_based_squared_error import (
    WeightedProbabilityBasedSquaredError,
    WeightedProbabilityBasedSquaredErrorOption,
)
from quara.loss_function.weighted_relative_entropy import (
    WeightedRelativeEntropy,
    WeightedRelativeEntropyOption,
)
from quara.minimization_algorithm.projected_gradient_descent_backtracking import (
    ProjectedGradientDescentBacktracking,
    ProjectedGradientDescentBacktrackingOption,
)


from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import (
    LossMinimizationEstimator,
)
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

from quara.simulation.standard_qtomography_simulation import NoiseSetting, TestSetting
from quara.simulation.standard_qtomography_simulation_flow import execute_simulation_test_settings

In [2]:
%reload_ext autoreload
%autoreload 2

# Settings

In [3]:
c_sys = generate_composite_system("qubit", 1)

In [4]:
# Generate settings for simulation
case_names = [
    "Linear (True)",
    "Linear (False)",
    "ProjectedLinear (True)",
    "ProjectedLinear (False)",
    "Maximum-Likelihood (True)",
    "Maximum-Likelihood (False)",
    "Least Squares (True)",
    "Least Squares (False)",
]

parametrizations = [True, False, True, False, True, False, True, False]

estimators = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
]

loss_list = [
    (None, None),
    (None, None),
    (None, None),
    (None, None),
    (WeightedRelativeEntropy(3), WeightedRelativeEntropyOption("identity")),
    (WeightedRelativeEntropy(4), WeightedRelativeEntropyOption("identity")),
    (
        WeightedProbabilityBasedSquaredError(3),
        WeightedProbabilityBasedSquaredErrorOption("identity"),
    ),
    (
        WeightedProbabilityBasedSquaredError(4),
        WeightedProbabilityBasedSquaredErrorOption("identity"),
    ),
]


def generate_pgdb_algo_option():
    return ProjectedGradientDescentBacktrackingOption(
        mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable",
        num_history_stopping_criterion_gradient_descent=1,
    )


algo_list = [
    (None, None),
    (None, None),
    (None, None),
    (None, None),
    (ProjectedGradientDescentBacktracking(), generate_pgdb_algo_option()),
    (ProjectedGradientDescentBacktracking(), generate_pgdb_algo_option()),
    (ProjectedGradientDescentBacktracking(), generate_pgdb_algo_option()),
    (ProjectedGradientDescentBacktracking(), generate_pgdb_algo_option()),
]

In [5]:
# Generate TestSetting 0: random_effective_lindbladian
# True Object
true_object_noise_setting = NoiseSetting(
    qoperation_base=("state", "z0"),
    method="random_effective_lindbladian",
    para={
        "lindbladian_base": "identity",
        "strength_h_part": 0.1,
        "strength_k_part": 0.1,
    },
)

# Tester Object
tester_names = [("povm", name) for name in ["x", "y", "z"]]
tester_object_noise_settings = [
    NoiseSetting(
        qoperation_base=name,
        method="random_effective_lindbladian",
        para={
            "lindbladian_base": "identity",
            "strength_h_part": 0.1,
            "strength_k_part": 0.1,
        },
    )
    for name in tester_names
]


# Test Setting
test_setting_0 = TestSetting(
    true_object=true_object_noise_setting,
    tester_objects=tester_object_noise_settings,
    seed=777,
    n_sample=2,
    n_rep=10,
    num_data=[10, 100],
    schedules="all",
    case_names=case_names,
    estimators=estimators,
    algo_list=algo_list,
    loss_list=loss_list,
    parametrizations=parametrizations,
    c_sys=c_sys,
)

# Generate TestSetting 1: depolarized
# True Object
true_object_noise_setting = NoiseSetting(
   qoperation_base=("state", "z0"), method="depolarized", para={"error_rate": 0.1,},
)
# Tester Object
tester_object_noise_settings = [
   NoiseSetting(qoperation_base=name, method="depolarized", para={"error_rate": 0.1,},)
   for name in tester_names
]

# Test Setting
test_setting_1 = TestSetting(
    true_object=true_object_noise_setting,
    tester_objects=tester_object_noise_settings,
    seed=777,
    n_sample=2,
    n_rep=10,
    num_data=[10, 100],
    schedules="all",
    case_names=case_names,
    estimators=estimators,
    algo_list=algo_list,
    loss_list=loss_list,
    parametrizations=parametrizations,
    c_sys=c_sys,
)

In [6]:
test_settings = [test_setting_0, test_setting_1]

# Execute

In [7]:
root_dir = "result_random_qst"
all_results = execute_simulation_test_settings(test_settings, root_dir)

/Users/tomoko/project/rcast/workspace/quara/quara/utils/matrix_util.py:195: ComplexWarning: Casting complex values to real discards the imaginary part
  tmp_hs = tmp_hs.astype(np.float64)
/Users/tomoko/project/rcast/workspace/quara/quara/objects/effective_lindbladian.py:723: ComplexWarning: Casting complex values to real discards the imaginary part
  tmp_hs = tmp_hs.astype(np.float64)
  0%|          | 0/10 [00:00<?, ?it/s]

Completed to write test_setting. result_random_qst/0/test_setting.pickle
Case 0: Linear (True)


 60%|██████    | 6/10 [00:00<00:00, 52.30it/s]

========== Summary ============
Name: Linear (True)
MSE of Empirical Distributions: OK
Consistency: OK (to_be_checked=False)
MSE of estimators: OK
Physicality Violation: OK
Case 1: Linear (False)
========== Summary ============
Name: Linear (False)
MSE of Empirical Distributions: OK
Consistency: OK (to_be_checked=False)
MSE of estimators: OK
Physicality Violation: OK
Case 2: ProjectedLinear (True)


100%|██████████| 10/10 [00:00<00:00, 17382.11it/s]

...

========== Summary ============
Name: Maximum-Likelihood (False)
MSE of Empirical Distributions: OK
Consistency: NG (to_be_checked=False)
MSE of estimators: OK
Physicality Violation: OK
Case 6: Least Squares (True)


  0%|          | 0/10 [00:00<?, ?it/s]

========== Summary ============
Name: Least Squares (True)
MSE of Empirical Distributions: OK
Consistency: OK (to_be_checked=False)
MSE of estimators: OK
Physicality Violation: OK
Case 7: Least Squares (False)


100%|██████████| 10/10 [00:00<00:00, 4409.49it/s]


========== Summary ============
Name: Least Squares (False)
MSE of Empirical Distributions: OK
Consistency: OK (to_be_checked=False)
MSE of estimators: OK
Physicality Violation: OK
Completed to write csv. result_random_qst/1/1/check_result.csv
total_result=True
Completed to write csv. result_random_qst/1/check_result.csv
Completed to write csv. result_random_qst/check_result.csv
============== OK: 32 cases (0 warnings), NG: 0 cases  in 421.3s (0:07:01)s==============
